In [2]:
from ipynb.fs.full.boundaryCellDisplacement import *

In [ ]:
class boundaryCellTraction(boundaryCellDisplacement, A):

    def __init__(self, edges, xy):
        
        sw_S, sw_N, sw_W, sw_E = 1, 1, 1, 1

        for edge in edges:
            if edge == "b": sw_S = 0
            if edge == "t": sw_N = 0
            if edge == "l": sw_W = 0
            if edge == "r": sw_E = 0
        
        if transient:
            self.a_P = (rho*dx*dy/(dt**2)) + A(xy).a_N*sw_N + A(xy).a_S**sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W
        else:
            self.a_P = A(xy).a_N*sw_N + A(xy).a_S**sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W

        self.a_N = A(xy).a_N
        self.a_S = A(xy).a_S 
        self.a_E = A(xy).a_E 
        self.a_W = A(xy).a_W 

        # self.a_faces = A(xy).a_N*sw_N + A(xy).a_S*sw_S + A(xy).a_E*sw_E + A(xy).a_W*sw_W 

    def b_temp(U_old, U_old_old, k, xy):      
        return A.b_temp(U_old, U_old_old, k, xy)
    
    def b_diff(edges, k, xy, U_previous):

        if xy == "x":
            uv = "v"
        if xy == "y":
            uv = "u"
            
        N_term =(
                    Sfy*A.coef(xy, "N", uv)*(
                    (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k))
                    /dx)
                )
        S_term =(
                    Sfy*A.coef(xy, "S", uv)*(
                        (boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dx)
                ) 
        E_term =(
                    Sfx*A.coef(xy, "E", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NE", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SE", uv, U_previous, k))
                        /dy)
                ) 
        W_term =(
                    Sfx*A.coef(xy, "W", uv)*(
                        (boundaryCellDisplacement.corner(edges, "NW", uv, U_previous, k) - boundaryCellDisplacement.corner(edges, "SW", uv, U_previous, k))
                        /dy)
                )

        for edge in edges:

            if (edge == "b") & (xy == "x") : S_term =  Sfy*tr_bottom_x  
            if (edge == "b") & (xy == "y") : S_term =  Sfy*tr_bottom_y  
            if (edge == "t") & (xy == "x") : N_term =  Sfy*tr_top_x  
            if (edge == "t") & (xy == "y") : N_term =  Sfy*tr_top_y  
            if (edge == "l") & (xy == "x") : W_term =  Sfx*tr_left_x  
            if (edge == "l") & (xy == "y") : W_term =  Sfx*tr_left_y  
            if (edge == "r") & (xy == "x") : E_term =  Sfx*tr_right_x  
            if (edge == "r") & (xy == "y") : E_term =  Sfx*tr_right_y        

        b_diffusion = (N_term + S_term + E_term + W_term)

        return b_diffusion

In [ ]:
def traction_cell_BCs(A_matrix, b_matrix, k, edges, xy, U_old, U_old_old, U_previous):

    A_matrix[k,k] = boundaryCellTraction(edges, xy).a_P

    b_matrix[k] =(
                    boundaryCellTraction.b_temp(U_old, U_old_old, k, xy)
                    +
                    boundaryCellTraction.b_diff(edges, k, xy, U_previous)  
                )
                
    for edge in edges:
        if edge == "b":
            #an
            A_matrix[k,index(k).n] = - boundaryCellTraction(edges, xy).a_N

        if edge == "t":
            #as
            A_matrix[k,index(k).s] = - boundaryCellTraction(edges, xy).a_S

        if edge == "l":
            #ae
            A_matrix[k,index(k).e] = - boundaryCellTraction(edges, xy).a_E

        if edge == "r":
            #aw
            A_matrix[k,index(k).w] = - boundaryCellTraction(edges, xy).a_W           

    if len(edges) == 1:

        if (edges[0] == "b") | (edges[0] == "t") :
            #ae
            A_matrix[k, index(k).e] = - boundaryCellTraction(edges, xy).a_E
            #aw
            A_matrix[k, index(k).w] = - boundaryCellTraction(edges, xy).a_W

        if (edges[0] == "l") | (edges[0] == "r") :
            #an
            A_matrix[k,index(k).n] = - boundaryCellTraction(edges, xy).a_N            
            #as
            A_matrix[k,index(k).s] = - boundaryCellTraction(edges, xy).a_S

    return A_matrix, b_matrix